In [2]:
from data_loader import load_data

In [23]:
import numpy as np
import random

class GeneticAlgorithm:
    def __init__(self, num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix,
                 population_size=100, generations=500, mutation_rate=0.1, crossover_rate=0.8):
        self.num_depots = num_depots
        self.num_customers = num_customers
        self.depot_capacities = np.array(depot_capacities)
        self.setup_costs = np.array(setup_costs)
        self.customer_demands = np.array(customer_demands)
        self.cost_matrix = np.array(cost_matrix)
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

    def initialize_population(self):
        # Each individual is a list of integers, where index is the customer and value is the depot assignment
        return [np.random.randint(0, self.num_depots, size=self.num_customers) for _ in range(self.population_size)]

    def fitness(self, individual):
        # Calculate the total cost for a given depot-customer assignment
        total_cost = 0
        depot_loads = np.zeros(self.num_depots)

        for customer, depot in enumerate(individual):
            depot_loads[depot] += self.customer_demands[customer]
            total_cost += self.cost_matrix[customer][depot]

        # Add depot setup costs for depots that are used
        used_depots = depot_loads > 0
        total_cost += np.sum(self.setup_costs[used_depots])

        # Penalize solutions that exceed depot capacities
        capacity_excess = np.maximum(0, depot_loads - self.depot_capacities)
        penalty = np.sum(capacity_excess) * 1e4  # Ceza katsayısını 1e4'e düşürdük
        total_cost += penalty

        return total_cost

    def selection(self, population, fitness_scores):
        # Turnuva Seçimi
        tournament_size = 5
        selected = random.sample(list(zip(population, fitness_scores)), tournament_size)
        selected = sorted(selected, key=lambda x: x[1])  # Fitness'e göre sırala
        return selected[0][0], selected[1][0]
    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            # Single-point crossover
            point = random.randint(1, self.num_customers - 1)
            child1 = np.concatenate((parent1[:point], parent2[point:]))
            child2 = np.concatenate((parent2[:point], parent1[point:]))
            return child1, child2
        return parent1.copy(), parent2.copy()

    def mutate(self, individual):
        if random.random() < self.mutation_rate:
            # Swap mutasyonu: İki müşterinin depot atamasını değiştir
            customer1, customer2 = random.sample(range(self.num_customers), 2)
            individual[customer1], individual[customer2] = individual[customer2], individual[customer1]
        return individual

    def evolve(self):
        population = self.initialize_population()
        best_solution = None
        best_cost = float('inf')

        for generation in range(self.generations):
            fitness_scores = [self.fitness(individual) for individual in population]
            new_population = []

            for _ in range(self.population_size // 2):
                parent1, parent2 = self.selection(population, fitness_scores)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                new_population.extend([child1, child2])

            population = new_population

            # Track the best solution
            min_cost = min(fitness_scores)
            if min_cost < best_cost:
                best_cost = min_cost
                best_solution = population[np.argmin(fitness_scores)]

            print(f"Generation {generation + 1}, Best Cost: {best_cost}")
        early_stop_generations = 10  # Son 50 nesilde iyileşme olmazsa durdur
        no_improvement_count = 0
        for generation in range(self.generations):
            fitness_scores = [self.fitness(individual) for individual in population]
            min_cost = min(fitness_scores)
            
            if min_cost < best_cost:
                best_cost = min_cost
                best_solution = population[np.argmin(fitness_scores)]
                no_improvement_count = 0
            else:
                no_improvement_count += 1
            
            if no_improvement_count >= early_stop_generations:
                print(f"Erken durdurma: İyileşme görülmedi. {generation + 1}. nesilde durduruldu.")
                break
        return best_solution, best_cost

In [30]:
# Load the problem data
file_path = "Dataset/wl_25"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)

# Initialize the genetic algorithm
ga = GeneticAlgorithm(
    num_depots=num_depots,
    num_customers=num_customers,
    depot_capacities=depot_capacities,
    setup_costs=setup_costs,
    customer_demands=customer_demands,
    cost_matrix=cost_matrix,
    population_size=1000,  # Adjust as needed
    generations=1000,  # Adjust as needed
    mutation_rate=0.5,  # Adjust as needed
    crossover_rate=1.0 # Adjust as needed
)

# Run the genetic algorithm
best_solution, best_cost = ga.evolve()

# Print the results
print("Best Solution (Depot Assignments):", best_solution)
print("Best Cost:", best_cost)

Generation 1, Best Cost: 1676698.425
Generation 2, Best Cost: 1645337.325
Generation 3, Best Cost: 1575141.1
Generation 4, Best Cost: 1562471.5999999999
Generation 5, Best Cost: 1499831.9374999998
Generation 6, Best Cost: 1393333.8250000002
Generation 7, Best Cost: 1367262.7500000002
Generation 8, Best Cost: 1318858.4
Generation 9, Best Cost: 1292260.8875
Generation 10, Best Cost: 1230706.9749999999
Generation 11, Best Cost: 1226178.1750000003
Generation 12, Best Cost: 1138596.5875000001
Generation 13, Best Cost: 1093839.7750000001
Generation 14, Best Cost: 1068860.625
Generation 15, Best Cost: 1057217.6250000002
Generation 16, Best Cost: 1045130.1625
Generation 17, Best Cost: 1026996.7625000001
Generation 18, Best Cost: 1002311.6125
Generation 19, Best Cost: 997195.525
Generation 20, Best Cost: 995497.175
Generation 21, Best Cost: 985534.1875000001
Generation 22, Best Cost: 969329.4125
Generation 23, Best Cost: 961254.4624999999
Generation 24, Best Cost: 953187.0750000001
Generation 2

In [29]:
# Load the problem data
file_path = "Dataset/wl_50"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)

# Initialize the genetic algorithm
ga = GeneticAlgorithm(
    num_depots=num_depots,
    num_customers=num_customers,
    depot_capacities=depot_capacities,
    setup_costs=setup_costs,
    customer_demands=customer_demands,
    cost_matrix=cost_matrix,
    population_size=1500,  # Adjust as needed
    generations=1500,  # Adjust as needed
    mutation_rate=0.5,  # Adjust as needed
    crossover_rate=1.0 # Adjust as needed
)

# Run the genetic algorithm
best_solution, best_cost = ga.evolve()

# Print the results
print("Best Solution (Depot Assignments):", best_solution)
print("Best Cost:", best_cost)

Generation 1, Best Cost: 2009833.525
Generation 2, Best Cost: 1900495.9250000003
Generation 3, Best Cost: 1890083.3749999998
Generation 4, Best Cost: 1890083.3749999998
Generation 5, Best Cost: 1730063.3500000003
Generation 6, Best Cost: 1684865.7375
Generation 7, Best Cost: 1660591.8125000002
Generation 8, Best Cost: 1633537.9375
Generation 9, Best Cost: 1608837.625
Generation 10, Best Cost: 1515517.9625
Generation 11, Best Cost: 1468721.7000000002
Generation 12, Best Cost: 1468721.7000000002
Generation 13, Best Cost: 1463593.1749999998
Generation 14, Best Cost: 1376666.6749999998
Generation 15, Best Cost: 1360273.7000000002
Generation 16, Best Cost: 1329286.6125
Generation 17, Best Cost: 1286274.275
Generation 18, Best Cost: 1262174.1374999997
Generation 19, Best Cost: 1256213.975
Generation 20, Best Cost: 1213869.2374999998
Generation 21, Best Cost: 1196846.35
Generation 22, Best Cost: 1189233.15
Generation 23, Best Cost: 1183815.1375
Generation 24, Best Cost: 1177126.1
Generation 2

In [28]:
# Load the problem data
file_path = "Dataset/wl_200"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)

# Initialize the genetic algorithm
ga = GeneticAlgorithm(
    num_depots=num_depots,
    num_customers=num_customers,
    depot_capacities=depot_capacities,
    setup_costs=setup_costs,
    customer_demands=customer_demands,
    cost_matrix=cost_matrix,
    population_size=1500,  # Adjust as needed
    generations=1500,  # Adjust as needed
    mutation_rate=0.5,  # Adjust as needed
    crossover_rate=1.0 # Adjust as needed
)

# Run the genetic algorithm
best_solution, best_cost = ga.evolve()

# Print the results
print("Best Solution (Depot Assignments):", best_solution)
print("Best Cost:", best_cost)

Generation 1, Best Cost: 42467.044250000006
Generation 2, Best Cost: 42287.16167
Generation 3, Best Cost: 41619.62097
Generation 4, Best Cost: 41619.62097
Generation 5, Best Cost: 41619.62097
Generation 6, Best Cost: 40315.53582
Generation 7, Best Cost: 40315.53582
Generation 8, Best Cost: 40315.53582
Generation 9, Best Cost: 40315.53582
Generation 10, Best Cost: 38686.60554999999
Generation 11, Best Cost: 38686.60554999999
Generation 12, Best Cost: 38686.60554999999
Generation 13, Best Cost: 38686.60554999999
Generation 14, Best Cost: 38261.256850000005
Generation 15, Best Cost: 38261.256850000005
Generation 16, Best Cost: 38198.1782
Generation 17, Best Cost: 37984.10910000001
Generation 18, Best Cost: 37984.10910000001
Generation 19, Best Cost: 37290.88295
Generation 20, Best Cost: 36091.0414
Generation 21, Best Cost: 35901.97229
Generation 22, Best Cost: 35901.97229
Generation 23, Best Cost: 35303.907849999996
Generation 24, Best Cost: 34574.028379999996
Generation 25, Best Cost: 34

In [27]:
# Load the problem data
file_path = "Dataset/wl_300"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)

# Initialize the genetic algorithm
ga = GeneticAlgorithm(
    num_depots=num_depots,
    num_customers=num_customers,
    depot_capacities=depot_capacities,
    setup_costs=setup_costs,
    customer_demands=customer_demands,
    cost_matrix=cost_matrix,
    population_size=1500,  # Adjust as needed
    generations=1500,  # Adjust as needed
    mutation_rate=0.5,  # Adjust as needed
    crossover_rate=1.0 # Adjust as needed
)

# Run the genetic algorithm
best_solution, best_cost = ga.evolve()

# Print the results
print("Best Solution (Depot Assignments):", best_solution)
print("Best Cost:", best_cost)

Generation 1, Best Cost: 97867.59573999999
Generation 2, Best Cost: 97867.59573999999
Generation 3, Best Cost: 97867.59573999999
Generation 4, Best Cost: 97105.42914000001
Generation 5, Best Cost: 96775.71834000002
Generation 6, Best Cost: 96775.71834000002
Generation 7, Best Cost: 95817.5345
Generation 8, Best Cost: 95817.5345
Generation 9, Best Cost: 95152.31368
Generation 10, Best Cost: 92368.15441
Generation 11, Best Cost: 92368.15441
Generation 12, Best Cost: 92368.15441
Generation 13, Best Cost: 92368.15441
Generation 14, Best Cost: 92231.66131
Generation 15, Best Cost: 92231.66131
Generation 16, Best Cost: 91423.37462
Generation 17, Best Cost: 89510.46311000001
Generation 18, Best Cost: 89510.46311000001
Generation 19, Best Cost: 89503.15926999999
Generation 20, Best Cost: 88672.92319000002
Generation 21, Best Cost: 88574.34296
Generation 22, Best Cost: 87164.54057000001
Generation 23, Best Cost: 87164.54057000001
Generation 24, Best Cost: 84789.24309
Generation 25, Best Cost: 8

In [26]:
# Load the problem data
file_path = "Dataset/wl_500"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)

# Initialize the genetic algorithm
ga = GeneticAlgorithm(
    num_depots=num_depots,
    num_customers=num_customers,
    depot_capacities=depot_capacities,
    setup_costs=setup_costs,
    customer_demands=customer_demands,
    cost_matrix=cost_matrix,
    population_size=1500,  # Adjust as needed
    generations=1500,  # Adjust as needed
    mutation_rate=0.5,  # Adjust as needed
    crossover_rate=1.0 # Adjust as needed
)

# Run the genetic algorithm
best_solution, best_cost = ga.evolve()

# Print the results
print("Best Solution (Depot Assignments):", best_solution)
print("Best Cost:", best_cost)

Generation 1, Best Cost: 99550.60376
Generation 2, Best Cost: 97702.71695
Generation 3, Best Cost: 97318.99359999999
Generation 4, Best Cost: 97318.99359999999
Generation 5, Best Cost: 97206.17187
Generation 6, Best Cost: 95399.06698000002
Generation 7, Best Cost: 95399.06698000002
Generation 8, Best Cost: 94769.87121
Generation 9, Best Cost: 94769.87121
Generation 10, Best Cost: 94520.09739000001
Generation 11, Best Cost: 93902.00792
Generation 12, Best Cost: 93496.78053
Generation 13, Best Cost: 93009.35253999999
Generation 14, Best Cost: 92935.64704000001
Generation 15, Best Cost: 90494.93374000001
Generation 16, Best Cost: 90494.93374000001
Generation 17, Best Cost: 90494.93374000001
Generation 18, Best Cost: 89854.32658
Generation 19, Best Cost: 89854.32658
Generation 20, Best Cost: 89854.32658
Generation 21, Best Cost: 89100.5941
Generation 22, Best Cost: 88403.24973
Generation 23, Best Cost: 86450.52757
Generation 24, Best Cost: 86450.52757
Generation 25, Best Cost: 86359.22562
